In [31]:
import numpy as np
import cma
import re
import math

x0 = 0.3
y0 = 0.3 
theta0 = math.pi
delta_t = 0.01

input_seq = [
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],
                [1,1],[1,1],[1,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0.3],[1,0.3],
                [1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[0.3,1],
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]
            ]

In [15]:
def generate_path(L, r):
    x = x0
    y = y0
    theta = thetha0

    path = [[x,y,theta]]

    for wl, wr in input_seq:
        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x + v * delta_t * math.cos(theta)
        y = y + v * delta_t * math.sin(theta)
        theta = theta + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        path.append([x,y,theta])
    
    return path

In [16]:
def loss(params):
    real = generate_path(real_params[0], real_params[1])
    pred = generate_path(params[0], params[1])
    return np.sqrt(np.mean((np.array(pred) - np.array(real)) ** 2))

In [17]:
def dist2target(path,target=[0,0]):
    return [np.linalg.norm(target)-np.array(step[0:-1])) for step in path]

In [18]:
L = 0.8
r = 0.69
real_params = [L,r]

L_fake = 0.6
r_fake = 0.13
fake_params = [L_fake,r_fake]

test_loss = loss(real_params)


In [19]:
es = cma.CMAEvolutionStrategy(len(real_params) * [0], 0.5, {'verbose': -3})
es.optimize(loss)

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 7.188720072169344e-02 1.0e+00 4.39e-01  3e-01  5e-01 0:00.0
    2     12 2.755491942895244e-02 1.4e+00 7.14e-01  6e-01  7e-01 0:00.0
    3     18 1.811749335266345e-02 1.3e+00 7.32e-01  6e-01  6e-01 0:00.0
  100    600 2.623307939999283e-13 4.7e+00 5.82e-08  2e-12  7e-12 0:00.3


In [20]:
print(f'Best loss value: {loss(es.result.xbest):.2}')

Best loss value: 1.1e-13


In [21]:
print('The parameters set is: L = {}, r = {}'.format(es.result.xbest[0],es.result.xbest[1]))


The parameters set is: L = 0.800000000001304, r = 0.6900000000009019


In [1]:
# Alternative with full control 
import numpy as np
import cma
import re
import math

x0 = 0.3
y0 = 0.3 
theta0 = math.pi
delta_t = 0.01

input_seq = [
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],
                [1,1],[1,1],[1,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0.3],[1,0.3],
                [1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[0.3,1],
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]
            ]

def generate_mpc_history(r, input_seq):
    x = x0
    y = y0
    theta = theta0

    hist = []
    elem = {'x':[x,y,theta]}
    for wl, wr in input_seq:
        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x + v * delta_t * math.cos(theta)
        y = y + v * delta_t * math.sin(theta)
        theta = theta + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        elem['u'] = [wl, wr]
        elem['final_x'] = [x,y,theta]
        hist.append(elem)
        elem = {'x':[x,y,theta]}
    return hist

def generate_step_history(r, mpc_history):
    hist = []
    for elem in mpc_history:
        x0, y0, theta0 = elem['x']
        wl, wr = elem['u']

        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x0 + v * delta_t * math.cos(theta0)
        y = y0 + v * delta_t * math.sin(theta0)
        theta = theta0 + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        hist.append([x,y,theta])
    return hist

def get_mean_sigma(param_set):
    min_param = min(param_set)
    max_param = max(param_set)

    mean = (min_param+max_param)/2
    sigma = (max_param-min_param)/4

    return mean, sigma

def loss(param_val): # {0.12, 0.16, 0.17}
    step_hist = generate_step_history(param_val[0], mpc_history)
    return np.sqrt(np.mean((np.array(step_mpc_history) - np.array(step_hist)) ** 2))

In [2]:
PARAMS_NUM = 1
SCENARIOS_NUM = 3

L = 0.5
r_set = [0.12, 0.16, 0.17]

true_r = 0.15

mean, sigma = get_mean_sigma(r_set)
print(mean, sigma)

0.14500000000000002 0.012500000000000004


In [3]:
mpc_history = generate_mpc_history(true_r,input_seq)
step_mpc_history = [elem['final_x'] for elem in mpc_history]

es = cma.CMAEvolutionStrategy((PARAMS_NUM+1)*[mean], sigma, {'verbose': -3,'popsize': SCENARIOS_NUM})
while not es.stop() and es.best.f > 1e-5:
    solutions = es.ask()
    es.tell(solutions, [loss(param_val) for param_val in solutions])
    es.disp(20)
print('terminated on ' + str(es.stop()))


Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      3 1.449235829722768e-04 1.0e+00 1.40e-02  1e-02  2e-02 0:00.0
    2      6 8.558900575653492e-05 1.2e+00 1.25e-02  1e-02  1e-02 0:00.0
    3      9 2.337905165729445e-04 1.3e+00 1.25e-02  1e-02  1e-02 0:00.0
terminated on {}


In [4]:
print('The parameters set is: r = {}, _ = {}'.format(es.result.xbest[0],es.result.xbest[1]))
print(es.result)

The parameters set is: r = 0.1499908428537282, _ = 0.17017737118540063
CMAEvolutionStrategyResult(xbest=array([0.14999084, 0.17017737]), fbest=8.636392277672802e-08, evals_best=40, evaluations=42, iterations=14, xfavorite=array([0.14999084, 0.17017737]), stds=array([0.00500316, 0.01035153]), stop={})


In [5]:
new_param_set = [sample[0] for sample in es.ask()]
print(new_param_set)

[0.14612622640926962, 0.1522213363839518, 0.14621410027617812]


In [6]:
# With the wrapper
from cma_es_wrapper import CMA_ES_Wrapper

true_r = 0.15
r_set = [0.12, 0.16, 0.17]# ====> RL1, RL2, RL3

#Do MPC   ======> hist [x, u, xf]

mpc_history = generate_mpc_history(true_r,input_seq)

es_wrapper = CMA_ES_Wrapper() # =======> new_param_set [r3, r4, r5]

# result = es_wrapper.optimize(r_set, mpc_history)
# print(result)

for _ in range(10):
    r_set = es_wrapper.optimize_step(r_set, mpc_history)
    print(r_set)



[0.145683954711632, 0.15099593094826286, 0.15168382596948632]
[0.1485199307056774, 0.1491782178365471, 0.15054209398141907]
[0.14852491117039493, 0.14888779709917505, 0.14917726826444525]
[0.14911040662427102, 0.1492268832623835, 0.14979017848931206]
[0.14951547094274978, 0.1495380683400996, 0.14982955607971926]
[0.14966578198560054, 0.14972113914075913, 0.14975779322522395]
[0.14972921434408296, 0.14973983885320621, 0.14975121324105528]
[0.14974660564218734, 0.1497653149091594, 0.1497718342422402]
[0.14976770662524555, 0.14977259894657372, 0.14978117300216567]
[0.14977224962010394, 0.1497753848465669, 0.14977672677570478]


In [ ]:
experiment.axle_lengths